# Usage of PyGromosTools on HPC-cluster


Note that you need support for the new `PyGromosTools` submission system shipped with `relase3`.

***
**Note**:
This notebook only works, if you have the LSF submission system on the machine you are unsing it on!
***

In [ ]:
# necessary imports are the Gromos_System and convenience functions emin and md
from pygromos.simulations.hpc_queuing.submission_systems import lsf
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.simulations.modules.preset_simulation_modules import emin, md

submission_system = lsf.LSF(nmpi=3) # this is for IBMs job queue!

## Initialization
Set up path to binaries and instantiate an initial `Gromos_System` object

In [ ]:
# binaries
gromosPP = None
gromosXX = None

# folders and title
system_name = "menthol-dmf-example"
work_folder = f"examples/example_files/{system_name}"

# files
in_cnf_path = f"example_files/submission-system-files/{system_name}-all-atom_54a7.cnf"
in_top_path = f"example_files/submission-system-files/{system_name}-all-atom_54a7.top"

# system
system = Gromos_System(
    work_folder, 
    system_name, 
    in_top_path=in_top_path, 
    in_cnf_path=in_cnf_path,
    in_gromosPP_bin_dir=gromosPP,
    in_gromosXX_bin_dir=gromosXX,
    submission_system=submission_system,
    verbose=False
)

## Energy minimization

Take advantage of the new submission system: convenience functions such as `emin` take in a parametrized `Gromos_System` object, automatically set up the calculation and return a new `Gromos_System` object with updated file paths.

In [ ]:
# run energy minimization
minimized_system = emin(system)

### Print out the new file paths

Note how the new `Gromos_System` object has new file paths associated with it.

In [ ]:
print(minimized_system.imd.path)
print(minimized_system.top.path)
print(minimized_system.cnf.path)

### Visualize the minimized system

In [ ]:
minimized_system.cnf.view

**Note:** The new `Gromos_System` object has an .imd file associated (from the energy minimization). Using the object for a subsequent calculation requires the .imd to be reset (as shown below) or a new `Imd` object to be passed to the function `md`. Otherwise `md` will be based on the previous (emin) input file.

In [ ]:
# Reset the old imd from emin 
minimized_system.imd = None

## Equilibration followed by subsequent production runs

In [ ]:
md_system = md(minimized_system, equilibration_runs=1, simulation_runs=2)

Again, note how the new `Gromos_System` object has new file paths associated with it.

In [ ]:
print(md_system.imd.path)
print(md_system.top.path)
print(md_system.cnf.path)

### Visualize the last configuration

In [ ]:
md_system.cnf.view